<a href="https://colab.research.google.com/github/eyaler/constrained/blob/main/pan3d/pan3d_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let our optima combine!

### A tutorial on combinatorial optimization with OR-Tools CP-SAT

or

### Can any form (diamond, pyramid) contain all ~~26~~ 27 letters?

[[Word Ways Challanges (Part 1)](https://digitalcommons.butler.edu/cgi/viewcontent.cgi?article=2326&context=wordways) (1979)]


In [1]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

from IPython.display import IFrame

IFrame(src='https://eyalgruss.com/constrained/pan3d/demo#אבגדהוזחטיךכלםמןנסעףפץצקרשת', width=1600, height=900)

In [2]:
#@title Install dependencies

!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=

In [3]:
#@title Download wordlist

!wget -nc https://github.com/eyaler/hebrew_wordlists/raw/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv -O wordlist.csv

--2024-10-28 14:33:28--  https://github.com/eyaler/hebrew_wordlists/raw/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/eyaler/hebrew_wordlists/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv [following]
--2024-10-28 14:33:29--  https://raw.githubusercontent.com/eyaler/hebrew_wordlists/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3574389 (3.4M) [text/plain]
Saving to: ‘wordlist.csv’

wordlist.csv        100%[===================>]   3.41M  --.-KB/s    in 0.1s    

2024-10-28 14:

In [15]:
#@title Load and preprocess wordlist

with open('wordlist.csv', encoding='utf8') as f:
    wordlist = f.read().replace('ך', 'כ').replace('ם', 'מ').replace('ן', 'נ').replace('ף', 'פ').replace('ץ', 'צ').splitlines()

word2score = {w.split(',')[0]: int(w.split(',')[1]) for w in wordlist[::-1]}

for w, s in list(word2score.items())[-5:]:
  print(f'{w} {s:,}')

print()
for w, s in list(word2score.items())[:5]:
  print(f'{w} {s:,}')

עמ 23,385,649
לא 37,013,163
על 51,535,935
את 69,835,058
של 74,641,565

תתרשלנה 1
תתרקמו 1
תתרפטנה 1
תתרפטי 1
תתרנותי 1


In [16]:
#@title Filter wordlist

limit = 2000

wordlist = [w for w in word2score if '"' not in w and "'" not in w and len(w) == 3 and (len(set(w)) == 3 or len(set(w)) == 2 and any(w.count(c) == 2 for c in 'כמנפצ'))][::-1][:limit]
print(f'{len(wordlist)=}')

print()
for w in wordlist[:5]:
  print(f'{w} {word2score[w]:,}')

print()
for w in wordlist[-5:]:
  print(f'{w} {word2score[w]:,}')

len(wordlist)=2000

הוא 21,755,267
היא 12,740,845
אני 11,308,587
אבל 9,421,249
בינ 8,466,909

פוג 1,907
יאט 1,898
אגש 1,892
סכה 1,883
מחא 1,882


In [6]:
#@title Setup model and solver

from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

#solver.parameters.log_search_progress = True  # If you want logging. See: https://d-krupke.github.io/cpsat-primer/05_parameters.html#logging

In [7]:
#@title Create cell (char) variables

cells = {}
flat = []
for x in range(3):
    for y in range(3):
        for z in range(3):
            cells[(x, y, z)] = model.new_int_var(0, 21, f'cells_{x}_{y}_{z}')
            flat.append(cells[(x, y, z)])

In [8]:
#@title Create boolean indicator variables, link them to the cell variables, and add count constraints

final_vals = (10, 12, 13, 16, 17)  # צ ,פ ,נ, מ ,כ

is_char = {}
for i in range(22):
    is_char[i] = []

    for j in range(27):
        is_char[i].append(model.new_bool_var(f'is_char_{i}_{j}'))
        model.add(flat[j] == i).only_enforce_if(is_char[i][j])  # https://developers.google.com/optimization/cp/channeling
        model.add(flat[j] != i).only_enforce_if(~is_char[i][j])

    if i in final_vals:
        model.add(sum(is_char[i]) == 2)
    else:
        model.add_exactly_one(is_char[i])

In [17]:
#@title Add allowed assignments (words + scores)
allowed = [tuple(ord(c) - ord('א') - sum(ord(c) >= ord(f) for f in 'כמנפצ') for c in w) + tuple([word2score[w]]) for w in wordlist]
for a in allowed[:5]:
  print(a)

score_values = [word2score[w] for w in wordlist]
domain = cp_model.Domain.from_values(score_values)
scores = []


def add_score_var():
    scores.append(model.new_int_var_from_domain(domain, f'score_{len(scores)}'))
    return scores[-1]


for i in range(3):
    for j in range(3):
        model.add_allowed_assignments((cells[(0, i, j)], cells[(1, i, j)], cells[(2, i, j)], add_score_var()), allowed)
        model.add_allowed_assignments((cells[(i, 0, j)], cells[(i, 1, j)], cells[(i, 2, j)], add_score_var()), allowed)
        model.add_allowed_assignments((cells[(i, j, 0)], cells[(i, j, 1)], cells[(i, j, 2)], add_score_var()), allowed)

(4, 5, 0, 21755267)
(4, 9, 0, 12740845)
(0, 13, 9, 11308587)
(0, 1, 11, 9421249)
(1, 9, 13, 8466909)


In [10]:
#@title Add sweetheart name constraint (this also helps break solution symmetry, and will run faster for this tutorial)

model.add(cells[(1, 1, 0)] == 9)   # י
model.add(cells[(1, 1, 1)] == 15)  # ע
model.add(cells[(1, 1, 2)] == 11)  # ל

In [11]:
#@title Solve to check whether any solution exists (optional)

solver.solve(model)
print(solver.status_name())

OPTIMAL


In [12]:
#@title Solve to find optimal solution score

class ObjectiveLogger(cp_model.CpSolverSolutionCallback):
    def on_solution_callback(self):
        print(f'{self.objective_value=} {self.best_objective_bound=}')

worst = model.new_int_var_from_domain(domain, 'worst')
model.add_min_equality(worst, scores)
model.maximize(worst)

solver.solve(model, ObjectiveLogger())

print([w for w in wordlist if word2score[w] == solver.objective_value])

self.objective_value=2127.0 self.best_objective_bound=229540.0
self.objective_value=2565.0 self.best_objective_bound=229540.0
self.objective_value=3269.0 self.best_objective_bound=229540.0
['צבנ']


In [13]:
#@title Solve to find all optimal solutions (of optimal scores)

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        super().__init__()
        self.solutions = []

    def decode(self, a, b, c):
        return ''.join(chr(self.value(v) + ord('א') + sum(self.value(v) >= f for f in final_vals)) for v in (a, b, c))

    def on_solution_callback(self):
        solution = []
        for i in range(3):
            for j in range(3):
                solution.append(self.decode(cells[(0, i, j)], cells[(1, i, j)], cells[(2, i, j)]))
                solution.append(self.decode(cells[(i, 0, j)], cells[(i, 1, j)], cells[(i, 2, j)]))
                solution.append(self.decode(cells[(i, j, 0)], cells[(i, j, 1)], cells[(i, j, 2)]))

        self.solutions.append(solution)
        print(len(self.solutions), solution)


model.clear_objective()  # needed for reusing the model to find multiple solution (i asked them to add this function!)
model.add(worst == int(solver.objective_value))  # comment out to get all (non-optimal) solutions

solver.parameters.enumerate_all_solutions = True  # Note this disables parallelism
solution_collector = SolutionCollector()
solver.solve(model, solution_collector)

1 ['שטנ', 'שאג', 'שחק', 'חרפ', 'חמד', 'אמצ', 'קפצ', 'קצת', 'גדת', 'איכ', 'טיס', 'טרפ', 'מעז', 'רעמ', 'יעל', 'צלב', 'פלכ', 'סמכ', 'גסה', 'נכה', 'נפצ', 'דמו', 'פזו', 'כזב', 'תכנ', 'צבנ', 'הונ']
2 ['שאג', 'שטנ', 'שחק', 'חמד', 'חרפ', 'טרפ', 'קצת', 'קפצ', 'נפצ', 'טיס', 'איכ', 'אמצ', 'רעמ', 'מעז', 'יעל', 'פלכ', 'צלב', 'כזב', 'נכה', 'גסה', 'גדת', 'פזו', 'דמו', 'סמכ', 'צבנ', 'תכנ', 'הונ']


4

In [14]:
from IPython.display import IFrame

chars = ''.join(solution_collector.solutions[0][::3])[::-1].replace('כ', 'ך', 1).replace('מ', 'ם', 1).replace('נ', 'ן', 1).replace('פ', 'ף', 1).replace('צ', 'ץ', 1)[::-1]
print(chars)
IFrame(src=f'https://eyalgruss.com/constrained/pan3d/demo#{chars}', width=1600, height=900)

שטנחרפקףצאיכמעזץלבגסהדםותךן
